# Assigning Emissions and Profit to Crops

## Import relevant libraries and load data

In [144]:
import numpy as np
import pandas as pd

In [145]:
# Read in data

resources_data = pd.read_excel("../data/interim/MasterSyntheticDatabase_v1.xlsx", header=0, usecols="B:M")

In [146]:
# Show first 5 rows

resources_data.head()
resources_data["Country"].unique()

array(['Ecuador', 'Brazil', 'Peru', 'Colombia', 'Suriname'], dtype=object)

In [147]:
# Remove datapoints with gaps
print(len(resources_data))
resources_data.drop(resources_data.index[resources_data['Worker Per Tonne'] == 0], inplace=True)
resources_data.drop(resources_data.index[np.isnan(resources_data['Worker Per Tonne'])], inplace=True)
print(len(resources_data))

1074
1042


## Assign emissions for each category based on real data from Brazil

In [148]:
# Show all categories present in data
categories = resources_data["Category"].unique()
print(categories)

['Ornamental & medicinal plants' 'Fruit' 'Vegetable' 'Herbs & Spices'
 'Cattle' 'Cocoa' 'Coffee' 'Oilseeds and oleaginous' 'Tea']


In [149]:
# Dictionary with info about CO2 emissions per tonne for each category

# Data from: https://www.sciencedirect.com/science/article/pii/S0959378018314365 (based on yearly average in Brazil)
# Direct data for: Vegetables, Fruits, Cattle, Oilseeds
# In data, "Other crops" includes: "cocoa, coffee, tea, and spices" - for us include also ornamental & medicinal plants in this category
# For us, include ornamental & medicinal plants in other

co2_per_tonne = {
    "Ornamental & medicinal plants" : 5.09e6,
    "Fruit" : 11.85e6,
    "Vegetable" : 11.85e6,
    "Herbs & Spices" : 5.09e6,
    "Cattle" : 466.30e6,
    "Cocoa" : 5.09e6,
    "Coffee" : 5.09e6,
    "Oilseeds and oleaginous" : 7.54e6,
    "Tea" : 5.09e6,
}


In [150]:
# Assign the CO2 on the actual dataset

for category in categories:
    resources_data.loc[resources_data["Category"] == category, "CO2 Per Tonne"] = co2_per_tonne[category]

resources_data.head()
    

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,NaN
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,NaN
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,NaN
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,NaN
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,NaN


In [151]:
# Find if there are any datapoints that have not been updated

resources_data.loc[np.isnan(resources_data['CO2 Per Tonne'])]

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne


## Assign prices for each crop based on real data from each country

### Brazil

In [152]:
# Show all crops present in Brazil
brazil_data = resources_data.loc[resources_data["Country"] == "Brazil"]
brazil_data.head()
crops = brazil_data["Crop"].unique()
print(crops)
print(len(crops))

['Açaí' 'Avocado' 'Banana' 'Cattle' 'Citrus' 'Coconut' 'Coffee Arabica'
 'Fig' 'Grapes' 'Guava' 'Lemon' 'Mango' 'Melon' 'Orange' 'Papaya' 'Pepper'
 'Watermelon' 'Yerba Mate']
18


In [153]:
# Dictionary with info about prices per tonne for Brazil (based on 2020)

# Data from: https://www.fao.org/faostat/en/#data/PP
# Indirect data for:
#  • Açaí (used prices of Brazil nuts),
#  • Cattle (live weight of meat, taken from 2016 data, as there is none recorded since)
#  • Yerba Mate (it is a type of tea, so used general Tea)

prices_brazil_USD_per_tonne = {
    "Açaí" : 577,
    "Avocado" : 344,
    "Banana" : 252,
    "Cattle" : 2782,
    "Citrus" : 216,
    "Coconut" : 91,
    "Coffee Arabica" : 1429,
    "Fig" : 1123,
    "Grapes" : 490,
    "Guava" : 217,
    "Lemon" : 216,
    "Mango" : 217,
    "Melon" : 197,
    "Orange" : 127,
    "Papaya" : 175,
    "Pepper" : 1803,
    "Watermelon" : 157,
    "Yerba Mate" : 157,
}
print(len(prices_brazil_USD_per_tonne))

18


In [154]:
# Assign the prices on the actual dataset

for crop in prices_brazil_USD_per_tonne:
    resources_data["Price Per Tonne"] = np.where(((resources_data["Crop"] == crop) & (resources_data["Country"] == "Brazil")), prices_brazil_USD_per_tonne[crop], resources_data["Price Per Tonne"])

resources_data


,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,NaN
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,577.0
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,NaN
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,NaN
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1069,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0
1070,Yerba Mate,Herbs & Spices,670.94,400.00,Brazil,NaN,1500000.0,1500.0000,8,0.005333,5090000.0,157.0
1071,Watermelon,Fruit,14608.13,1155.00,Brazil,NaN,32000000.0,32000.0000,824,0.025750,11850000.0,157.0
1072,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0


### Colombia

In [155]:
# Show all crops present in Colombia
colombia_data = resources_data.loc[resources_data["Country"] == "Colombia"]
colombia_data.head()
crops = colombia_data["Crop"].unique()
print(crops)
print(len(crops))

['Avocado' 'Banana' 'Basil' 'Bay Leaf' 'Blueberry' 'Cape gooseberry'
 'Cattle' 'Chive' 'Cocoa' 'Coffee Arabica' 'Flowers' 'Foliage' 'Guava'
 'Lemon' 'Mint' 'Oil Palm' 'Oregano' 'Other Ornamental plants' 'Papaya'
 'Passion Fruit' 'Pineapple' 'Plantain' 'Rose' 'Rose Hip' 'Rosemary'
 'Tarragon' 'Thyme']
27


In [156]:
# Dictionary with info about prices per tonne for Colombia (based on 2020)

# Data from: https://www.fao.org/faostat/en/#data/PP
# Indirect data for:
#  • All berries (used prices of Strawberries),
#  • Cattle (live weight of meat, taken from 2015 data)
#  • Cocoa (taken from 2015 data)
#  • All spices (taken from Peru 2020 data),
#  • Foliage (taken as Maize)
#  • Passion fruit (taken as peaches and nectarines)

prices_colombia_USD_per_tonne = {
    "Avocado" : 1186,
    "Banana" : 482,
    "Basil" : 3031,
    "Bay Leaf" : 3031,
    "Blueberry" : 1517,
    "Cape gooseberry" : 1517, 
    "Cattle" : 1290,
    "Chive" : 3031,
    "Cocoa" : 2588,
    "Coffee Arabica" : 4388,
    "Flowers" : 3031,
    "Foliage" : 365,
    "Guava" : 925,
    "Lemon" : 469,
    "Mint" : 1636,
    "Oil Palm" : 729,
    "Oregano" : 3031,
    "Other Ornamental plants" : 3031,
    "Papaya" : 411,
    "Passion Fruit" : 1156,
    "Pineapple" : 558,
    "Plantain" : 521,
    "Rose" : 3031,
    "Rose Hip" : 3031,
    "Rosemary" : 3031,
    "Tarragon" : 3031,
    "Thyme" : 3031,
}
print(len(prices_colombia_USD_per_tonne))

27


In [157]:
# Assign the prices on the actual dataset

for crop in prices_colombia_USD_per_tonne:
    resources_data["Price Per Tonne"] = np.where(((resources_data["Crop"] == crop) & (resources_data["Country"] == "Colombia")), prices_colombia_USD_per_tonne[crop], resources_data["Price Per Tonne"])
    
resources_data


,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,NaN
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,577.0
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,NaN
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,NaN
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1069,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0
1070,Yerba Mate,Herbs & Spices,670.94,400.00,Brazil,NaN,1500000.0,1500.0000,8,0.005333,5090000.0,157.0
1071,Watermelon,Fruit,14608.13,1155.00,Brazil,NaN,32000000.0,32000.0000,824,0.025750,11850000.0,157.0
1072,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0


### Ecuador

In [158]:
# Show all crops present in Ecuador

ecuador_data = resources_data.loc[resources_data["Country"] == "Ecuador"]
ecuador_data.head()
crops = ecuador_data["Crop"].unique()
print(crops)
print(len(crops))

['Abaca' 'Banana' 'Cocoa' 'Flowers' 'Mango' 'Oil Palm' 'Passion Fruit'
 'Pepper' 'Pineapple' 'Pitajaya' 'Plantain' 'Rose' 'Tea']
13


In [159]:
# Dictionary with info about prices per tonne for Ecuador (based on 2020)

# Data from: https://www.fao.org/faostat/en/#data/PP
# Indirect data for:
#  • Abaca (used prices from 2017),
#  • Flowers (taken from Peru 2020 spices data),
#  • Mango, Passion Fruit and Pitajaya (used prices from 2018 for Tropical Fruit),
#  • Pepper (used prices from 2017),
#  • Tea (used prices from 2017),

prices_ecuador_USD_per_tonne = {
    "Abaca" : 2030,
    "Banana" : 327,
    "Cocoa" : 2097,
    "Flowers" : 3031,
    "Mango" : 238,
    "Oil Palm" : 199,
    "Passion Fruit" : 238,
    "Pepper" : 1147,
    "Pineapple" : 478,
    "Pitajaya" : 238,
    "Plantain" : 204,
    "Rose" : 3031,
    "Tea" : 100,
}
print(len(prices_ecuador_USD_per_tonne))

13


In [160]:
# Assign the prices on the actual dataset

for crop in prices_ecuador_USD_per_tonne:
    resources_data["Price Per Tonne"] = np.where(((resources_data["Crop"] == crop) & (resources_data["Country"] == "Ecuador")), prices_ecuador_USD_per_tonne[crop], resources_data["Price Per Tonne"])
    
resources_data

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,2030.0
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,577.0
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,NaN
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,NaN
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1069,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0
1070,Yerba Mate,Herbs & Spices,670.94,400.00,Brazil,NaN,1500000.0,1500.0000,8,0.005333,5090000.0,157.0
1071,Watermelon,Fruit,14608.13,1155.00,Brazil,NaN,32000000.0,32000.0000,824,0.025750,11850000.0,157.0
1072,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0


### Peru

In [161]:
# Show all crops present in Guyana

peru_data = resources_data.loc[resources_data["Country"] == "Peru"]
peru_data.head()
crops = peru_data["Crop"].unique()
print(crops)
print(len(crops))

['Asparagus' 'Avocado' 'Banana' 'Blueberry' 'Cocoa' 'Coffee Arabica'
 'Ginger' 'Grapes' 'Lemon' 'Mandarin orange' 'Mango' 'Onion'
 'Passion Fruit' 'Pomegranate']
14


In [162]:
# Dictionary with info about prices per tonne for Peru (based on 2020)

# Data from: https://www.fao.org/faostat/en/#data/PP
# Indirect data for:
#  • Banana (taken from Ecuador - countries have similar Avocado prices),
#  • Blueberry (used prices for Strawberry),
#  • Ginger (used prices for garlic),
#  • Passion Fruit and Pomegranate (used prices from Peaches)

prices_peru_USD_per_tonne = {
    "Asparagus" : 893,
    "Avocado" : 986,
    "Banana" : 482,
    "Blueberry" : 705,
    "Cocoa" : 1965,
    "Coffee Arabica" : 1683,
    "Ginger" : 1532,
    "Grapes" : 729,
    "Lemon" : 305,
    "Mandarin orange" : 332,
    "Mango" : 275,
    "Onion" : 208,
    "Passion Fruit" : 545,
    "Pomegranate" : 545,
}
print(len(prices_peru_USD_per_tonne))

14


In [163]:
# Assign the prices on the actual dataset

for crop in prices_peru_USD_per_tonne:
    resources_data["Price Per Tonne"] = np.where(((resources_data["Crop"] == crop) & (resources_data["Country"] == "Peru")), prices_peru_USD_per_tonne[crop], resources_data["Price Per Tonne"])
    
resources_data

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,2030.0
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,577.0
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,893.0
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,893.0
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,893.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1069,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0
1070,Yerba Mate,Herbs & Spices,670.94,400.00,Brazil,NaN,1500000.0,1500.0000,8,0.005333,5090000.0,157.0
1071,Watermelon,Fruit,14608.13,1155.00,Brazil,NaN,32000000.0,32000.0000,824,0.025750,11850000.0,157.0
1072,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0


### Suriname

In [164]:
# Show all crops present in Suriname

suriname_data = resources_data.loc[resources_data["Country"] == "Suriname"]
suriname_data.head()
crops = suriname_data["Crop"].unique()
print(crops)

['Banana']


In [165]:
# Only crop is Banana, and cannot find data for Suriname, so just take data from Brazil

resources_data["Price Per Tonne"] = np.where(((resources_data["Crop"] == "Banana") & (resources_data["Country"] == "Suriname")), prices_brazil_USD_per_tonne["Banana"], resources_data["Price Per Tonne"])
    
resources_data

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne
0,Abaca,Ornamental & medicinal plants,2016.89,8.20,Ecuador,NaN,16501.0,16.5010,143,8.666141,5090000.0,2030.0
1,Açaí,Fruit,309.60,245.84,Brazil,NaN,161524.0,161.5240,2,0.012382,11850000.0,577.0
2,Asparagus,Vegetable,4351.63,431.87,Peru,NaN,5482000.0,5482.0000,5793,1.056731,11850000.0,893.0
3,Asparagus,Vegetable,170.00,105.00,Peru,NaN,926098.5,926.0985,97,0.104740,11850000.0,893.0
4,Asparagus,Vegetable,239.20,120.00,Peru,NaN,1080000.0,1080.0000,337,0.312037,11850000.0,893.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1069,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0
1070,Yerba Mate,Herbs & Spices,670.94,400.00,Brazil,NaN,1500000.0,1500.0000,8,0.005333,5090000.0,157.0
1071,Watermelon,Fruit,14608.13,1155.00,Brazil,NaN,32000000.0,32000.0000,824,0.025750,11850000.0,157.0
1072,Watermelon,Fruit,1849.76,200.00,Brazil,NaN,6048000.0,6048.0000,512,0.084656,11850000.0,157.0


## Save All New Data

In [166]:
# Find if there are any datapoints that have not been updated

resources_data.loc[np.isnan(resources_data['Price Per Tonne'])]

,Crop,Category,Total Hectares,Production Hectares,Country,Location,Volume Kilos,Volume Tonnes,Total Workers,Worker Per Tonne,CO2 Per Tonne,Price Per Tonne


In [168]:
resources_data.to_excel("../data/interim/MasterSyntheticDatabase_v2.xlsx")